In [133]:
import numpy as np
import gurobipy as gp
import networkx as nx
import pandas as pd
import cvxpy as cp
import copy
from parse import read_input_file, write_output_file, read_output_file
from utils import is_valid_network, average_pairwise_distance, average_pairwise_distance_fast
import sys
import matplotlib.pyplot as plt
import scipy as sp
from gurobipy import GRB
from IPython.display import display

In [306]:
def initialize_dsm(G):
    dsm = {}
    for i in G.nodes():
        dsm[i] = [G.degree(i), 0, 0]
    for x in G.edges(data=True):
        i = x[0]
        j = x[1]
        weight = x[2]['weight']
        dsm[i][1] = dsm[i][1]+weight
        dsm[j][1] = dsm[j][1]+weight
        dsm[i][2] = max(dsm[i][2], weight)
        dsm[j][2] = max(dsm[j][2], weight)
    return(dsm)   

def get_score_list(dsm, C1, C2, C3):
    score_list = {}
    for i in dsm.keys():
        score_list[i] = C1*dsm[i][0]+C2*dsm[i][0]/(dsm[i][1]+0.0001)+C3/(dsm[i][2]+0.001)
    return(score_list)

def find_min_starter(score_list):
    nodes = list(score_list.keys())
    weight = list(score_list.values())
    starter = nodes[weight.index(min(weight))]
    return(starter)

def find_max_starter(score_list):
    nodes = list(score_list.keys())
    weight = list(score_list.values())
    starter = nodes[weight.index(max(weight))]
    return(starter)

In [307]:
def initialize_w_cf_list(G):
    w_cf_list = {}
    nodes = G.nodes()
    for i in nodes:
        w_cf_list[i] = [1000000, 1000000, None, 1000000]
    return(w_cf_list)

def update_touchable_set(v, S):
    neighbors = set(G.neighbors(v))
    new_S = S.union(neighbors)
    new_S.add(v)
    return(new_S)

def update_w_cf_list(w_cf_list, v, T0, C4, C5):
    entries = w_cf_list[v].copy()
    neighbors = G.neighbors(v)
    score = w_cf_list[v][3]
    for i in neighbors:
        if i in T0:
            weight = G.get_edge_data(i, v)['weight']
            cf = w_cf_list[i][1]+weight
            new_score = C4*weight+C5*cf
            if new_score<score:
                entries[0] = weight
                entries[1] = cf
                entries[2] = i
                entries[3] = new_score
                score = new_score
    return(entries)

def get_priority_list(V, somelist, dsm, C6):
    priority = {}
    for i in V:
        priority[i] =somelist[i][3]+C6*dsm[i][0]
    return(priority)

def update_dsm(dsm, v):
    neighbors = G.neighbors(v)
    for i in neighbors:
        dsm[i][0] = dsm[i][0]-1
    return(dsm)

In [308]:
def create_subgraph(G, T, T0):
    if len(T0)==1:
        H = G.subgraph(list(T0)).copy()
        return(H)
    else:
        edges_subset = []
        for x in T:
            if None not in x:
                edges_subset.append(x)
        H = G.edge_subgraph(edges_subset).copy()
        return(H)

In [309]:
def find_opt_tree(G, C1, C2, C3, C4, C5, C6):
    n = len(G.nodes())
    dsm = initialize_dsm(G)
    score_list = get_score_list(dsm, C1, C2, C3)
    starter = find_max_starter(score_list)
    T = {(starter, None)}
    T0 = {starter}
    S = update_touchable_set(starter, set())
    V = set(G.nodes())
    V.remove(starter)
    w_cf_list = initialize_w_cf_list(G)
    w_cf_list[starter][1] = 0
    while len(S)<n:
        copy_w_cf_list = {}
        for v in V:
            copy_w_cf_list[v] = update_w_cf_list(w_cf_list, v, T0, C4, C5)
        priority_list = get_priority_list(V, copy_w_cf_list, dsm, C6)
        starter = find_min_starter(priority_list)
        entry = copy_w_cf_list[starter].copy()
        w_cf_list[starter] = entry
        parent = copy_w_cf_list[starter][2]
        S = update_touchable_set(starter, S)
        V.remove(starter)
        dsm = update_dsm(dsm, starter)
        T.add((starter, parent))
        T0.add(starter)
    H = create_subgraph(G, T, T0)
    return(H)

In [314]:
def solve_opt_tree(G):
    for x in G.nodes():
        if G.degree(x)>=n-1:
            return(G.subgraph(x).copy())
    A0 = 1000000000
    for c in np.arange(-20, 1, 1):
        for d in [(0.2, 0.2, 0.6), (0.2, 0.6, 0.2), (0.6, 0.2, 0.2)]:
            for e in [(0.9, 0.1), (1, 1)]:
                H = find_opt_tree(G, d[0], d[1], d[2], e[0], e[1], c)
                A = average_pairwise_distance_fast(H)
                if A<A0:
                    A0 = A
                    opt_tree = H.copy()
    return(opt_tree)

In [321]:
for i in range(1, 401):
    input_filename = 'inputs/large-'+str(i)+'.in'
    output_filename = 'outputs/large-'+str(i)+'.out'
    better_output_filename = 'better_outputs/large-'+str(i)+'.out'
    G = read_input_file(input_filename)
    H = solve_opt_tree(G)
    J = read_output_file(output_filename, G)
    if (average_pairwise_distance_fast(H)<=average_pairwise_distance_fast(J)):
        write_output_file(H, better_output_filename)
    else:
        write_output_file(J, better_output_filename)

In [322]:
for i in range(1, 304):
    input_filename = 'inputs/medium-'+str(i)+'.in'
    output_filename = 'outputs/medium-'+str(i)+'.out'
    better_output_filename = 'better_outputs/medium-'+str(i)+'.out'
    G = read_input_file(input_filename)
    H = solve_opt_tree(G)
    J = read_output_file(output_filename, G)
    if (average_pairwise_distance_fast(H)<=average_pairwise_distance_fast(J)):
        write_output_file(H, better_output_filename)
    else:
        write_output_file(J, better_output_filename)

In [323]:
for i in range(1, 304):
    input_filename = 'inputs/small-'+str(i)+'.in'
    output_filename = 'outputs/small-'+str(i)+'.out'
    better_output_filename = 'better_outputs/small-'+str(i)+'.out'
    G = read_input_file(input_filename)
    H = solve_opt_tree(G)
    J = read_output_file(output_filename, G)
    if (average_pairwise_distance_fast(H)<=average_pairwise_distance_fast(J)):
        write_output_file(H, better_output_filename)
    else:
        write_output_file(J, better_output_filename)